In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from siphon.catalog import TDSCatalog
import metpy.calc as mpcalc
from metpy.units import units
from scipy.ndimage import gaussian_filter

C:\Users\Tony\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
def time_dim(ds, var_name):
    possible_time_dims = ['time', 'time1', 'time2', 'time3']
    time_dim = None
    for dim in possible_time_dims:
        if dim in ds[var_name].dims:
            time_dim = dim
            break
    if time_dim is None:
        raise ValueError('Could not find the time dimension')
    return time_dim

def find_press_dim(ds, var_name):
    possible_iso_dims = ['isobaric', 'isobaric1', 'isobaric2', 'isobaric3']
    iso_dim = None
    for dim in possible_iso_dims:
        if dim in ds[var_name].dims:
            iso_dim = dim
            break
    if iso_dim is None:
        raise ValueError('Could not find the iso dimension')
    return iso_dim

def pull_data(directions):
    tds_gfs = TDSCatalog('https://thredds.ucar.edu/thredds/catalog/grib/NCEP/GFS/Global_0p25deg/latest.html')
    gfs_ds = tds_gfs.datasets[0]
    ds = xr.open_dataset(gfs_ds.access_urls['OPENDAP'])
    ds_sliced = ds.sel(lat=slice(directions['North'], directions['South']), lon=slice(directions['West'], directions['East']))
    z = ds['Geopotential_height_isobaric'].sel(isobaric=slice(50000, 70000))
    z_700 = ds['Geopotential_height_isobaric'].sel(isobaric=70000)
    temp = ds['Temperature_isobaric'].sel(isobaric=slice(50000, 70000))
    t_700 = ds['Temperature_isobaric'].sel(isobaric=70000)
    u = ds_sliced['u-component_of_wind_isobaric'].sel(isobaric=slice(50000, 100000))
    v = ds_sliced['v-component_of_wind_isobaric'].sel(isobaric=slice(50000, 100000))
    time = time_dim(ds, 'Geopotential_height_isobaric')
    iso_dim = find_press_dim(ds, 'Geopotential_height_isobaric')
    pressure_levels = z['isobaric'].values[::-1]
    return z, z_700, temp, t_700, time, iso_dim, pressure_levels, u, v

def calculate_qg(z, t_700, directions, time, iso_dim, pressure_levels, u, v):
    for i in range(0, 1):  # Loop over all times
        z_i = z.isel(**{time: i})
        u_i = u.isel(**{time: i})
        v_i = v.isel(**{time: i})
        temp_i = t_700.isel(**{time: i})

        # Select the geopotential heights at the 500 and 700 hPa levels and calculate the geostrophic wind
        z_500 = z_i.sel(isobaric=50000)
        z_700 = z_i.sel(isobaric=70000)
        u_g_500, v_g_500 = mpcalc.geostrophic_wind(z_500)
        V_g_500 = mpcalc.wind_speed(u_g_500, v_g_500)
        u_g_700, v_g_700 = mpcalc.geostrophic_wind(z_700)
        V_g_700 = mpcalc.wind_speed(u_g_700, v_g_700)

        # Calculate absolute vorticity and select the 500 and 700 hPa levels
        absolute_vorticity = mpcalc.absolute_vorticity(u_i, v_i)
        absolute_vorticity_700 = absolute_vorticity.sel(isobaric=70000)
        absolute_vorticity_500 = absolute_vorticity.sel(isobaric=50000)

        # Calculate the gradient of the absolute vorticity
        absolute_vorticity_gradient_500 = np.gradient(absolute_vorticity_500, axis=(-2, -1))
        absolute_vorticity_gradient_700 = np.gradient(absolute_vorticity_700, axis=(-2, -1))

        # Calculate the advection of absolute vorticity
        absolute_vorticity_advection_500 = -(V_g_500 * absolute_vorticity_gradient_500[0] + V_g_500 * absolute_vorticity_gradient_500[1])
        absolute_vorticity_advection_700 = -(V_g_700 * absolute_vorticity_gradient_700[0] + V_g_700 * absolute_vorticity_gradient_700[1])

        # Calculate the differential vorticity advection
        differential = (absolute_vorticity_advection_500 - absolute_vorticity_advection_700) / (z_500 - z_700)
        #pressure_gradient_advection = np.gradient(absolute_vorticity_advection_500, pressure_levels, axis=0)
        print(differential)



In [3]:
directions = {'North': 50, 
              'East': 250, 
              'South': 20, 
              'West': 220}

(z, z_700, temp, t_700, time, iso_dim, pressure_levels, u, v) = pull_data(directions)
calculate_qg(z, t_700, directions, time, iso_dim, pressure_levels, u, v)

C:\Users\Tony\anaconda3\lib\site-packages\siphon\catalog.py:292: UserWarning: URL https://thredds.ucar.edu/thredds/catalog/grib/NCEP/GFS/Global_0p25deg/latest.html returned HTML. Changing to: https://thredds.ucar.edu/thredds/catalog/grib/NCEP/GFS/Global_0p25deg/latest.xml
  warnings.warn('URL {} returned HTML. Changing to: {}'.format(self.catalog_url,
C:\Users\Tony\anaconda3\lib\site-packages\pint\facets\plain\quantity.py:998: RuntimeWarning: divide by zero encountered in divide
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)
C:\Users\Tony\anaconda3\lib\site-packages\pint\facets\plain\quantity.py:998: RuntimeWarning: invalid value encountered in multiply
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)
C:\Users\Tony\anaconda3\lib\site-packages\pint\facets\plain\quantity.py:998: RuntimeWarning: divide by zero encountered in divide
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)
C:\Users\Tony\anaconda3\lib\site-packages\pint\facets\pla

ValueError: operands could not be broadcast together with shapes (721,1440) (121,121) 